In [1]:
--Exploratory data analysis of alternative fuel stations
--Skills used: Aggregate Functions, CTEs, Temp Tables, Subqueries, Converting Data Types, creating Views

-- How many public stations are available? What percentage?

SELECT Status, 
    COUNT(Status) as Num_stations, 
    CAST(COUNT(Status) AS float)/CAST((SELECT COUNT(Status) from dbo.alt_fuel_stations_clean) AS float)*100 as Percent_stations  
    FROM dbo.alt_fuel_stations_clean
WHERE Access_code = 'public'
GROUP BY Status
ORDER BY Percent_stations DESC

(3 rows affected)

Total execution time: 00:00:00.052

Status,Num_stations,Percent_stations
Available,61604,86.53827243738323
Temporarily Unavailable,4216,5.922429657100313
Planned,166,0.23318864399398767


In [2]:
--What kind of stations are being planned? Which are being planned the most?
SELECT Fuel_type, 
    COUNT(Fuel_type) as Num_stations,
    CAST(COUNT(Fuel_type) AS float)/CAST((SELECT COUNT(Fuel_type) from dbo.alt_fuel_stations_clean) AS float)*100 as Percent_stations  
    FROM dbo.alt_fuel_stations_clean
GROUP BY Fuel_type
ORDER BY Num_stations DESC

(8 rows affected)

Total execution time: 00:00:00.090

Fuel_type,Num_stations,Percent_stations
Electric,60371,84.80621461783753
Ethanol (E85),4464,6.270807872223862
Propane (LPG),2859,4.016182729992836
Compressed Natural Gas,1451,2.0382935086462415
Biodiesel (B20 and above),1193,1.6758677848483572
Renewable Diesel (R20 and above),576,0.8091364996417886
Hydrogen,139,0.19526037057327883
Liquefied Natural Gas,134,0.18823661623611054


In [6]:
--Which states have available, public hydrogen fueling stations, and how many?
SELECT State, COUNT(State) as Num_Hyd_stations
    FROM dbo.alt_fuel_stations_clean
WHERE Fuel_type = 'Hydrogen' AND Status = 'Available' AND Access_code = 'public'
GROUP BY State
ORDER BY Num_Hyd_stations DESC

(2 rows affected)

Total execution time: 00:00:00.042

State,Num_Hyd_stations
CA,60
HI,1


In [7]:
--Drilling down: Looks like California has the most public, available hydrogen fueling stations by a lot - which three cities in CA have the most?
SELECT City, COUNT(City) as Num_Hyd_stations
    FROM dbo.alt_fuel_stations_clean
WHERE State = 'CA' AND Fuel_type = 'Hydrogen' AND Status = 'Available' AND Access_code = 'public'
GROUP BY City
ORDER BY Num_Hyd_stations DESC
OFFSET 0 ROWS FETCH FIRST 3 ROWS ONLY;

(3 rows affected)

Total execution time: 00:00:00.125

City,Num_Hyd_stations
Los Angeles,4
San Francisco,3
San Jose,3


In [11]:
--I thought that a city as large as LA would have more than 4 hydrogen fueling stations.
--Let's check how many they have that are planned, not available, or private
SELECT Status, Access_code, COUNT(*) as Num_Hyd_stations
    FROM dbo.alt_fuel_stations_clean
WHERE City = 'Los Angeles' and Fuel_type = 'Hydrogen' AND (Status = 'Planned' OR Status = 'Temporarily Unavailable')
GROUP BY Status, Access_code
ORDER BY Num_Hyd_stations DESC

(2 rows affected)

Total execution time: 00:00:00.046

Status,Access_code,Num_Hyd_stations
Planned,public,1
Temporarily Unavailable,public,1


In [12]:
--There appears to just not be that many hydrogen refueling stations, even in LA. How many electric fuel stations are there in LA?
SELECT Status, Access_code, COUNT(*) as Num_Elec_stations
    FROM dbo.alt_fuel_stations_clean
WHERE City = 'Los Angeles' AND Fuel_type = 'Electric'
GROUP BY Status, Access_code
ORDER BY Num_Elec_stations DESC

(4 rows affected)

Total execution time: 00:00:00.049

Status,Access_code,Num_Elec_stations
Available,public,1451
Temporarily Unavailable,public,143
Available,private,69
Planned,public,1


In [13]:
--There seem to be a lot of available, public electric fuel stations in LA -- what are the top 10 cities?
--Using a common table expression (CTE) to calculate the total number of electric stations per city

With Electric_CTE (City, State, Num_Elec_stations) AS (
    SELECT City, State, COUNT(*) AS Num_elec_stations FROM dbo.alt_fuel_stations_clean
    WHERE Fuel_type = 'Electric' AND Access_code = 'public' AND Status = 'Available'
    GROUP BY State, City
)

SELECT City, State, Num_Elec_stations FROM Electric_CTE
    ORDER BY Num_Elec_stations DESC
    OFFSET 0 ROWS FETCH FIRST 10 ROWS ONLY;

(10 rows affected)

Total execution time: 00:00:00.057

City,State,Num_Elec_stations
Los Angeles,CA,1450
San Diego,CA,651
Irvine,CA,619
Atlanta,GA,529
San Jose,CA,514
Austin,TX,494
Kansas City,MO,466
Boston,MA,406
San Francisco,CA,403
Seattle,WA,380


In [14]:
--Looks like most of the cities with the highest numbers of available, public electric fueling stations are in California
--What is the average per state?
--Using a temporary table:
DROP Table if exists #Elec_stations_by_State
Create Table #Elec_stations_by_State
(
    State nvarchar(255),
    Num_Elec_stations int,
)

Insert into #Elec_stations_by_State
SELECT State, COUNT(*) as Num_Elec_stations FROM dbo.alt_fuel_stations_clean
    WHERE Fuel_type = 'Electric' AND Access_code = 'public' AND Status = 'Available'
    GROUP BY State

Select * from #Elec_stations_by_State
    ORDER BY Num_Elec_stations DESC
    OFFSET 0 ROWS FETCH FIRST 10 ROWS ONLY;

(71 rows affected)

(10 rows affected)

Total execution time: 00:00:00.069

State,Num_Elec_stations
CA,14175
NY,3323
FL,2795
TX,2519
MA,2430
CO,1740
WA,1663
GA,1543
MD,1334
PA,1326


In [16]:
--Calculating average number of available, public electric stations per state:
SELECT AVG(Num_Elec_stations) as Average_Elec_Stations_Per_State FROM #Elec_stations_by_State

(1 row affected)

Total execution time: 00:00:00.008

Average_Elec_Stations_Per_State
739


In [21]:
--Creating a view to later visualize the locations of these stations using Tableau 
--containing only the valid Latitude and Longitude values
CREATE VIEW alt_fuel_station_locations AS
SELECT Fuel_type, Facility_type, Status, Access_code, City, State, ZIP, Country, Latitude, Longitude
    FROM dbo.alt_fuel_stations_clean
    WHERE Latitude != 1 AND Latitude != 0 AND Latitude !=-1 AND
    Longitude != 1 AND Longitude != 0 AND Longitude !=-1

Commands completed successfully.

Total execution time: 00:00:00.039